# IPEX-LLM：在 Intel GPU 上本地运行 BGE 嵌入模型

> [IPEX-LLM](https://github.com/intel-analytics/ipex-llm) 是一个 PyTorch 库，用于在 Intel CPU 和 GPU（例如，本地 PC 的 iGPU，以及 Arc、Flex 和 Max 等独立 GPU）上以极低的延迟运行 LLM。

本示例将介绍如何使用 LangChain，在 Intel GPU 上通过 `ipex-llm` 优化来执行嵌入任务。这将有助于实现 RAG、文档问答等应用。

> **注意**
>
> 建议只有拥有 Intel Arc A 系列 GPU（Intel Arc A300 系列或 Pro A60 除外）的 Windows 用户直接运行此 Jupyter notebook。对于其他情况（例如 Linux 用户、Intel iGPU 等），建议在终端中使用 Python 脚本运行代码以获得最佳体验。

## 安装先决条件
要在 Intel GPU 上利用 IPEX-LLM，需要完成几个工具安装和环境准备的先决步骤。

如果您是 Windows 用户，请访问 [在 Windows 和 Intel GPU 上安装 IPEX-LLM 指南](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_windows_gpu.md)，然后按照 [安装先决条件](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_windows_gpu.md#install-prerequisites) 更新 GPU 驱动程序（可选）并安装 Conda。

如果您是 Linux 用户，请访问 [在 Linux 和 Intel GPU 上安装 IPEX-LLM](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_linux_gpu.md)，然后按照 [**安装先决条件**](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Quickstart/install_linux_gpu.md#install-prerequisites) 安装 GPU 驱动程序、Intel® oneAPI Base Toolkit 2024.0 和 Conda。

## 设置

在安装完先决条件后，您应该已经创建了一个已安装所有先决条件的 conda 环境。**在此 conda 环境中启动 jupyter 服务**：

In [ ]:
%pip install -qU langchain langchain-community

安装 IPEX-LLM 以优化 Intel GPU，以及 `sentence-transformers`。

In [ ]:
%pip install --pre --upgrade ipex-llm[xpu] --extra-index-url https://pytorch-extension.intel.com/release-whl/stable/xpu/us/
%pip install sentence-transformers

> **注意**
>
> 你也可以使用 `https://pytorch-extension.intel.com/release-whl/stable/xpu/cn/` 作为 `extra-indel-url`。

## 运行时配置

为了达到最佳性能，建议根据你的设备设置几个环境变量：

### 适用于使用 Intel Core Ultra 集成 GPU 的 Windows 用户

In [ ]:
import os

os.environ["SYCL_CACHE_PERSISTENT"] = "1"
os.environ["BIGDL_LLM_XMX_DISABLED"] = "1"

### 适用于拥有 Intel Arc A 系列 GPU 的 Windows 用户

In [ ]:
import os

os.environ["SYCL_CACHE_PERSISTENT"] = "1"

> **注意**
>
> 首次在 Intel iGPU/Intel Arc A300 系列或 Pro A60 上运行模型时，编译可能需要几分钟时间。
>
> 对于其他 GPU 类型，Windows 用户请参考[此处](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Overview/install_gpu.md#runtime-configuration)，Linux 用户请参考[此处](https://github.com/intel-analytics/ipex-llm/blob/main/docs/mddocs/Overview/install_gpu.md#runtime-configuration-1)。

## 基本用法

在初始化 `IpexLLMBgeEmbeddings` 时，将 `model_kwargs` 中的 `device` 设置为 `"xpu"` 会将嵌入模型放在 Intel GPU 上，并受益于 IPEX-LLM 的优化：

In [ ]:
from langchain_community.embeddings import IpexLLMBgeEmbeddings

embedding_model = IpexLLMBgeEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs={"device": "xpu"},
    encode_kwargs={"normalize_embeddings": True},
)

API 参考
- [IpexLLMBgeEmbeddings](https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.ipex_llm.IpexLLMBgeEmbeddings.html)

In [ ]:
sentence = "IPEX-LLM is a PyTorch library for running LLM on Intel CPU and GPU (e.g., local PC with iGPU, discrete GPU such as Arc, Flex and Max) with very low latency."
query = "What is IPEX-LLM?"

text_embeddings = embedding_model.embed_documents([sentence, query])
print(f"text_embeddings[0][:10]: {text_embeddings[0][:10]}")
print(f"text_embeddings[1][:10]: {text_embeddings[1][:10]}")

query_embedding = embedding_model.embed_query(query)
print(f"query_embedding[:10]: {query_embedding[:10]}")